# Tutorial 11 - Signal Detection Experiment Simulation + Analysis

*Written and revised by Jozsef Arato, Mengfan Zhang, Dominik Pegler*  
Computational Cognition Course, University of Vienna  
https://github.com/univiemops/tewa1-computational-cognition

---
**This tutorial will cover:**

1.  simulate an experiment, based on the theoretical d-prime measures of signal detection theory.  

2.  Calcualte the behavioral responses in this experiment (0 -noise only, 1-signal present)

3. Calculate the empirical d' and criterion from the responses and the simulated stimuli  

4. compare the theoretical and empirical d-prime, see how it is affected by the number of trials

---

## Import libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

Set simulation main parameters

In [ ]:
signal_mean = 4
noise_mean = 0  # do not change this
sd = (
    1.5  # the assumption of sdT is that noise and signal have equal Standard Deviation,
)
crit = 2  # decision criteria, above which response is signal present

##
1.  Calculate and plot *theoretical* SDT predicitons

2. change the parameters above and see how this plot changes

stat.norm.pdf :  normal probability density function

In [ ]:
dprime_theory = (signalMean - noiseMean) / sd

x = np.linspace(-10, 15, 100)  #  excitation strength (hypothetical neural activity)
noise = stats.norm.pdf(x, noiseMean, sd)
signal = stats.norm.pdf(x, signalMean, sd)
plt.figure()
plt.plot(x, noise, label="noise Distribution")
plt.plot(x, signal, label="signal Distribution")
plt.xlabel("Neural Response Strength", fontsize=18)
plt.ylabel("Probability", fontsize=18)

plt.plot(
    [crit, crit], [0, np.max(signal)], color="k"
)  # fill out for vertical criterion line

plt.legend()
plt.title("d-prime: " + str(np.round(dprime_theory, 2)))

### Set up experiment

0 - noise trial
1 - signal trial

half of trials is signal, but in random order

In [ ]:
n_trial = 1000  # num of signal trials + num of noise trials
# Stimuli=# YOUR CODE
# YOUR CODE
# print(Stimuli[0:50])

In [ ]:
stimuli = np.random.permutation(
    np.concatenate((np.ones(int(n_trial / 2)), np.zeros(int(n_trial / 2))))
)
print(stimuli[0:50])
np.sum(stimuli == 1)

In [ ]:
stimuli = np.zeros(n_trial)
stimuli[np.random.choice(np.arange(n_trial), int(n_trial / 2), replace=False)] = 1
print(stimuli[0:50])
np.sum(stimuli == 1)

alternative solution

In [ ]:
stimuli = np.random.binomial(1, 0.5, size=n_trial)
np.sum(stimuli == 1)


### Simulate assumed response variable in the participants brain

using the same paramters, as defined above


vectorized solution

In [ ]:
neural_response = np.zeros(n_trial)  # set up empty array

neural_response[stimuli == 1] = np.random.normal(
    signal_mean, sd, int(n_trial / 2)
)  # simulated neural response, when the stimulus is noise
neural_response[stimuli == 0] = np.random.normal(
    noise_mean, sd, int(n_trial / 2)
)  # simulated neural response, when the stimulus is signal

print(neural_response[0:50])

for loop solution

In [ ]:
neural_response = np.zeros(n_trial)
for tr in range(n_trial):
    if stimuli[tr] == 1:
        neural_response[tr] = np.random.normal(signal_mean, sd)
    else:
        neural_response[tr] = np.random.normal(noise_mean, sd)

In [ ]:
plt.hist(neural_response)

### simualte/calculate behavioral responses, based on neural signal and criteria

0 = response is noise
1 = response is signal

for loop solution

In [ ]:
response = np.zeros(n_trial)  # YOUR CODE
for tr in range(n_trial):
    if Neuralresponse[tr] > crit:
        response[tr] = 1
    else:
        response[tr] = 0

vectorized solution

In [ ]:
response = np.zeros(1000)
response[Neuralresponse > crit] = 1

## Now the "experiment" is ready, we can start to analyze it. :-)

##
based on the stimulus and response vectors,  calculate the number of:

1. misses
2. hits
3. false alarms
4. correct rejections


tip: use np.sum

In [ ]:
hit = np.sum((stimuli == 1) & (response == 1))
miss = np.sum((stimuli == 1) & (response == 0))
fa = np.sum((stimuli == 0) & (response == 1))
corr_rej = np.sum((stimuli == 0) & (response == 0))

print("Num hits: ", hit)
print("misses : ", miss)
print("Num False Alarms: ", fa)
print("Num Correct Rejection: ", corr_rej)

alternatively, solve with for cycle

In [ ]:
hit = 0
miss = 0
fa = 0
corr_rej = 0
for tr in range(len(stimuli)):
    if stimuli[tr] == 1 and response[tr] == 1:
        hit += 1
    if stimuli[tr] == 1 and response[tr] == 0:
        miss += 1
    if stimuli[tr] == 0 and response[tr] == 1:
        fa += 1
    if stimuli[tr] == 0 and response[tr] == 0:
        corr_rej += 1
print("Num hits: ", hit)
print("misses : ", miss)
print("Num False Alarms: ", fa)
print("Num Correct Rejection: ", corr_rej)

Calculate the hit-rate and false alarm rate from the above

In [ ]:
hit_rate = hit / (hit + miss)
fa_rate = fa / (fa + corr_rej)
print("hit_rate", hit_rate)
print("False Alarm Rate", fa_rate)

##  Calculate Empirical Signal Detection Theory Measures
 see the meaning of inverse normal cdf here: [link text](http://gru.stanford.edu/lib/exe/fetch.php/tutorials/zof1.png)


Sensitivity
d'=  invnorm(H)-invnorm(FA)

Bias:
c= -(invnorm(H)+invnorm(FA))/2



for inverse cumulative normal we can use:

stats.norm.ppf


In [ ]:
dprime_emp = stats.norm.ppf(hit_rate) - stats.norm.ppf(fa_rate)
bias = (stats.norm.ppf(hit_rate) + stats.norm.ppf(fa_rate)) / 2
print("D-prime from Data: ", np.round(dprime_emp, 2))
print("bias from Data: ", np.round(bias, 2))

In [ ]:
stats.norm.ppf(hit_rate)

## Homework
Take all the necessary code from above, re-use it in the function below:
The function should take 4 input parameters:
1. MeanSignal: mean of signal distribution
2. SD (same for signal and noise)
3. Crit criterion (above which response is signal)
4. NTr : number of trials (half of which is signal, half noise)

The function should perform the simulation as above and return the  Hit Rate and False Alarm Rate

In [ ]:
def sim_exp(MeanSignal, SD, Crit, NTr):
    # YOUR CODE
    # YOUR CODE
    # YOUR CODE
    # YOUR CODE
    return hit_rate, fa_rate

 ## Homework 2.:
calling the function above, repeatedly, we can make an ROC curve (that shows the relationship of Hit rate and false alarm rate).
1. keep MeanSignal fixed and change the criterion in a for loop (for the values defined below)
2. visualize  with scatter plot: False alarm Rate on x-axis, Hit rate on y axis
3. do not forget to make the plot nice with legends, labels, fontsize, etc


In [ ]:
criteria = np.linspace(1, 3, 8)
# YOUR CODE
# YOUR CODE
# YOUR CODE
# YOUR CODE

## Homework 3.:
similarly to the task above, but change both the criterion and the SignalMean (2 embedded for loops, to make multiple ROC curves), and make a scatter plot with the result!

use at least 8 values for criterion and 4 values for SignalMean.
The 4 values for Signal Mean should be above 0 and below 12.


dots that have the same signal mean (but different Criteria),should show up in the same color.
dots that have different signal mean, should have different colors!    (this will make the figure readable)

do not forget to make the plot nice with legends, labels, fontsize, etc




In [ ]:
# YOUR CODE
# YOUR CODE
# YOUR CODE
# YOUR CODE

#  Psyhcometric curves

In [ ]:
stim = np.random.normal(0, 2, 80)
ans = 1 / (1 + np.exp(-stim))
plt.scatter(stim, ans)
plt.xticks([])
plt.yticks(fontsize=16)
plt.xlabel("Luminance", fontsize=18)
plt.ylabel("p(Yes)", fontsize=18)

In [ ]:
def sigm_func(x, c1, c2):
    return 1 / (1 + np.exp(-c1 * (x - c2)))


stim = np.linspace(-4, 4, 80)
ans = sigm_func(stim, 1, 0)
ans2 = sigm_func(stim, 2, 0)
ans3 = sigm_func(stim, 0.5, 0)
ans4 = sigm_func(stim, 1, 2)

plt.plot(stim, ans, linewidth=3, label="Shape(c1)=1")
plt.plot(stim, ans2, linewidth=3, label="Shape(c1)=2")
plt.plot(stim, ans3, linewidth=3, label="Shape(c1)=0.5")
plt.plot(stim, ans4, linewidth=3, label="Shape(c1)=1, c2=1")

plt.xticks([])
plt.yticks(fontsize=16)
plt.xlabel("Luminance", fontsize=18)
plt.ylabel("p(Yes)", fontsize=18)
plt.legend()

# let's try to simulate an experiment based on a sigmoid curve

the idea is that we used simulate N trials fro each stimulus strength level, where the probabily of 'yes' response is determined by the sigmoid curve

In [ ]:
xs = np.linspace(-3, 3, 10)  # stimulus range of values
# sigm_func(np.linspace(-3,3,20),1,0)
ntr = 8  # num of trials at each stimulus strength
ans_sim = np.zeros((len(xs), ntr))  #  array for storing responses
for cx, x in enumerate(xs):  # loop though and count values of xs
    ptr = sigm_func(x, 1.3, 0.02)  # probabilty of response 1
    for n in range(ntr):
        rand_p = np.random.rand()  # random value (0-1 range)
        if rand_p < ptr:  # if random number is smaller than probability, answer is 1
            ans_sim[cx, n] = 1
        else:  # other wise answer is zero (in fact this part could be cut)
            ans_sim[cx, n] = 0

plt.scatter(xs, np.mean(ans_sim, 1))

# plt.plot(xs,sigm_func(xs,c1,0),linewidth=3,label='Shape(c1)'+str(c1))

plt.yticks(fontsize=16)
plt.xticks(fontsize=16)
plt.xlabel("Stimulus Strength", fontsize=18)
plt.ylabel("p(Yes)", fontsize=18)

## Trying values for Psychomteric function shape parameter manually

In [ ]:
plt.figure(figsize=(12, 7))
# plt.plot(xs,np.mean(ans_sim,1))##
for cc, c1 in enumerate([0.5, 1, 2, 3]):
    plt.subplot(2, 2, cc + 1)
    plt.scatter(xs, np.mean(ans_sim, 1))

    plt.plot(xs, sigm_func(xs, c1, 0), linewidth=3, label="Shape(c1)" + str(c1))

    plt.yticks(fontsize=13)
    plt.xticks(fontsize=13)
    plt.xlabel("Stimulus Strength", fontsize=15)
    plt.ylabel("p(Yes)", fontsize=15)

    plt.title("c1=" + str(c1) + " c2=0", fontsize=17)
plt.tight_layout()

## sigmoid likelihood function for answers 0 and 1

(log likelihood for computational reasons)

In [ ]:
xx = np.repeat(xs, ntr)  # array for X

ans_true = ans_sim.flatten()  # 1d array for Y

ps = sigm_func(xx, 1, 0)
ll = np.sum(np.log(ps[ans_true == 1])) + np.sum(np.log(1 - ps[ans_true == 0]))


def sigm_l_lfit(pars):
    ps = sigm_func(xx, pars[0], pars[1])
    ll = np.sum(np.log(ps[ans_true == 1])) + np.sum(np.log(1 - ps[ans_true == 0]))
    return -ll

## maximum likelihood model fitting for psychometric curve

In [ ]:
from scipy.optimize import minimize

mod = minimize(sigm_l_lfit, x0=[0.5, 1])

plt.scatter(xs, np.mean(ans_sim, 1))

plt.plot(
    xs, sigm_func(xs, mod.x[0], mod.x[1]), linewidth=3, label="Shape(c1)" + str(c1)
)

plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.xlabel("Stimulus Strength", fontsize=15)
plt.ylabel("p(Yes)", fontsize=15)

plt.title(
    "c1=" + str(np.round(mod.x[0], 2)) + " c2=" + str(np.round(mod.x[1], 2)),
    fontsize=17,
)

# logistic regression for the same data

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg = logistic_regression()

In [ ]:
log_reg.fit(xx.reshape(-1, 1), ans_sim.reshape(-1, 1))

In [ ]:
log_reg.coef_

In [ ]:
log_reg.intercept_